In [22]:
import pandas as pd
import numpy as np

In [23]:
train_csv = pd.read_csv("train_images.csv")

In [ ]:
print(((train_csv['label']).value_counts()).describe())
len(train_csv['label'].unique())

DATA PREPROCESSING

    Resize images (e.g., 224x224).
    Normalize pixel values to range [0, 1].
    Apply data augmentation (rotation, flipping, etc.). 

In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:

def preprocess_data(csv_data, base_folder='', img_size=(224, 224)):
    """
    Preprocess images based on CSV paths and labels.
    Args:
        csv_data: Pandas DataFrame with 'path' and 'label' columns.
        base_folder: Base folder containing the images (e.g., "train_images").
        img_size: Target size for resizing images.
    Returns:
        images: Numpy array of image data.
        labels: Numpy array of labels.
    """
    images = []
    labels = []
    for _, row in csv_data.iterrows():
        img_path = (base_folder + row[0])  # Add base folder to path
        label = row[1]  # Label

        # Load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize
        images.append(img_array)
        labels.append(label)
    
    return np.array(images), np.array(labels)


# Preprocess training data
train_images, train_labels = preprocess_data(train_csv, base_folder="C:/Users/martj/Desktop/Applied ML/AML-assignment/train_images")

# Convert labels to one-hot encoding
num_classes = len(np.unique(train_labels))

# # Adjust labels to 0-based indexing
train_labels = train_labels - 1
train_labels = to_categorical(train_labels, num_classes=200)


C:\Users\martj\AppData\Local\Temp\ipykernel_516\2724898715.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_path = (base_folder + row[0])  # Add base folder to path
C:\Users\martj\AppData\Local\Temp\ipykernel_516\2724898715.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = row[1]  # Label


In [70]:
# Find unique one-hot encoded vectors
unique_vectors = np.unique(train_labels, axis=0)
#print(len(unique_vectors))  # This will give the number of unique vectors


In [27]:
# First, split into training + validation and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Next, split the training data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.25, random_state=42, stratify=train_labels
)  # 0.25 x 0.8 = 0.2 of the original data

In [28]:
print("train data size", len(train_labels))
print("test data size", len(test_labels))
print("validation data size", len(val_labels))

train data size 2355
test data size 786
validation data size 785


In [29]:
# Find unique one-hot encoded vectors
unique_vectors = np.unique(train_labels, axis=0)
print("unique labels in training data", len(unique_vectors))  # This will give the number of unique vectors
# Find unique one-hot encoded vectors
unique_vectors = np.unique(test_labels, axis=0)
print("unique labels in test data", len(unique_vectors))  # This will give the number of unique vectors
# Find unique one-hot encoded vectors
unique_vectors = np.unique(val_labels, axis=0)
print("unique labels in validation data", len(unique_vectors))  # This will give the number of unique vectors

unique labels in training data 200
unique labels in test data 200
unique labels in validation data 200


**Define data generators**

In [30]:
####Can mess with this too, although I did not see much change when doing so 
train_datagen = ImageDataGenerator(
    #Randomly rotates the image within a range of ±20 degrees
    rotation_range=30,
    #Shifts the image horizontally by a random fraction (up to 20% of the width)
    width_shift_range=0.3,
    #Shifts the image vertically by a random fraction (up to 20% of the height)
    height_shift_range=0.3,
    #Applies a random shearing transformation within 0.2 radians
    shear_range=0.2,
    #Zooms in or out on the image randomly by up to 20%
    zoom_range=0.2,
    # Randomly flips the image horizontally
    horizontal_flip=True,
    #Fills in pixels that are generated due to transformations using the nearest pixel values
    fill_mode='nearest'
)
# Apply data augmentation to training images only
train_generator = train_datagen.flow(
    train_images, train_labels, batch_size=32
)
val_datagen = ImageDataGenerator()  # No augmentation for validation
val_generator = val_datagen.flow(
    val_images, val_labels, batch_size=32
)


**Define the model**

In [35]:
#### This is implementation only for MobileNetV2 model ####


# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
# from tensorflow.keras.regularizers import l2
# # Load Pre-trained Model
# # Maybe look for different pretrained models as well
# base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
# # Add Custom Layers
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# # First dense layer
# x = Dense(128, activation='relu')(x)
# predictions = Dense(num_classes, activation='softmax')(x)
# # Define the Model
# model = Model(inputs=base_model.input, outputs=predictions)
# #Freeze Base Layers for Transfer Learning
# for layer in base_model.layers:
#     layer.trainable = False
# # base_model.trainable = True
# # for layer in base_model.layers[:-20]:  # Example: Freeze all but the last 20 layers
# #     layer.trainable = False
# # Compile Model
# #smaller learning rate when fine-tuning to avoid destroying pre-trained weights
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [33]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras import layers, Model

# Load pretrained MobileNetV2
mobile_net = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
mobile_net.trainable = False  # Freeze the base model

# Load pretrained EfficientNetB0
efficient_net = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
efficient_net.trainable = False  # Freeze the base model

# Shared input (a single input for both models)
input_layer = tf.keras.Input(shape=(224, 224, 3))

# Pass the shared input through both models
mobile_net_output = mobile_net(input_layer)
mobile_net_output = layers.GlobalAveragePooling2D()(mobile_net_output)

efficient_net_output = efficient_net(input_layer)
efficient_net_output = layers.GlobalAveragePooling2D()(efficient_net_output)

# Combine outputs
combined = layers.Concatenate()([mobile_net_output, efficient_net_output])

# Add classifier layers
#dense_layer = layers.Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.01))(combined)
dropout_layer = layers.Dropout(0.6)(combined)
output_layer = layers.Dense(200, activation="softmax",kernel_regularizer=tf.keras.regularizers.l2(0.01))(dropout_layer)  # Assume 200 classes

# Define the combined model
model = Model(inputs=input_layer, outputs=output_layer)

efficient_net.trainable = True  # Unfreeze EfficientNetB0
for layer in efficient_net.layers[:100]:  # Freeze the first 100 layers
    layer.trainable = False

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Model summary
model.summary()


**Train model**

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_images) // 32,
    validation_steps=len(val_images) // 32
)
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, batch_size=32)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
model.save('bird_species_classifier.h5')



In [ ]:
print(train_labels.shape)  # Should be (number of samples,)
print(val_labels.shape)    # Should be (number of samples,)


In [ ]:
from tensorflow.keras.models import load_model
# Load the model from the H5 file
model = load_model('bird_species_classifier.h5')
# Check the model's architecture or summary
model.summary()


TEST IMAGES


In [ ]:
test_csv = pd.read_csv("test_images_path.csv")
test_csv = test_csv.drop(columns=['id'])
test_images, test_labels = preprocess_data(test_csv, base_folder="C:/Users/martj/Desktop/Applied ML/AML-assignment/test_images")

In [ ]:
print(((test_csv['label']).value_counts()).describe())
len(test_csv['label'].unique())
#train_csv.count()

In [112]:
predictions = model.predict(test_images)

125/125 [==============================] - 130s 1s/step


In [82]:
len(predictions)
has_nan = np.isnan(predictions).any()
has_nan

False

In [ ]:
# Decode the predicted class for each sample
predicted_classes = np.argmax(predictions, axis=1)

# `predicted_classes` will now contain the class index for each sample
print(type(predicted_classes))
predicted_classes = predicted_classes +1

In [114]:
# Count occurrences of each class
unique_classes, class_counts = np.unique(predicted_classes, return_counts=True)

In [ ]:
# Print the counts for each class
for class_label, count in zip(unique_classes, class_counts):
    print(f"Class {class_label}: {count} predictions")

In [ ]:
# Step 3: Replace the 'label' column
test_csv['label'] = predicted_classes

# Step 4: Save the updated DataFrame to a new CSV file
test_csv.to_csv("test_images_path_updated.csv", index=False)

print("Updated CSV saved as 'test_images_path_updated.csv'.")

In [ ]:
import pandas as pd

# Define the function to process the CSV
def clean_image_path(csv_input_path, csv_output_path):
    """
    Cleans the `image_path` column in the CSV file by extracting only the image ID.
    
    Args:
    - csv_input_path (str): Path to the input CSV file.
    - csv_output_path (str): Path to save the cleaned CSV file.
    """
    # Load the CSV
    df = pd.read_csv(csv_input_path)

    # Extract the image ID from the `image_path` column
    df['image_path'] = df['image_path'].str.extract(r'/(\d+)\.jpg')[0]

    # Save the cleaned DataFrame
    df.to_csv(csv_output_path, index=False)
    print(f"Cleaned CSV saved at: {csv_output_path}")

# Example usage
csv_input_path = "test_images_path_updated.csv"  # Input CSV path
csv_output_path = "test_images_path_cleaned.csv"  # Output CSV path
clean_image_path(csv_input_path, csv_output_path)
